# Tiền xử lý Bộ Dữ liệu
Trong notebook này, mục tiêu là xử lý bộ dữ liệu sao cho sẵn sàng để sử dụng cho quá trình huấn luyện. Để đạt được mục tiêu này, các bước sau sẽ được thực hiện trong notebook:
1. Kết hợp các tệp CSV của bộ dữ liệu thành một bộ dữ liệu duy nhất
2. Phân tích sơ bộ bộ dữ liệu
3. Làm sạch dữ liệu – xử lý giá trị thiếu, bản ghi trùng lặp, v.v.
4. Xử lý sâu hơn, bao gồm giải quyết vấn đề mất cân bằng lớp nghiêm trọng và đổi tên các cột (nếu cần thiết)
5. Lưu bộ dữ liệu đã xử lý

Lưu ý: Notebook sẽ được chia thành hai phần phụ, trong đó bộ dữ liệu huấn luyện và bộ dữ liệu kiểm tra sẽ được xử lý riêng biệt. Tuy nhiên, các bước thực hiện sẽ giống nhau đối với cả hai bộ dữ liệu.

In [45]:
# import the packages to be used
import os       # Tạo thư mục và xóa các tệp
import csv      # Loại bỏ các cột thừa trong bộ dữ liệu CSE-CIC-IDS2018
import numpy as np
import pandas as pd
import random

# Đặt hạt giống ngẫu nhiên để đảm bảo kết quả có thể tái tạo được
random.seed(10)
np.random.seed(10)

# CIC-IDS2017 Dataset

## Bước 1. Kết hợp các tệp của bộ dữ liệu

Trong bước này, chúng ta cũng sẽ xử lý một vấn đề khi các tệp của bộ dữ liệu CIC-IDS2017 bao gồm ký tự thay thế (U+FFFD). Nguyên nhân là do bộ dữ liệu sử dụng ký tự "–" (mã Unicode: U+2013), mà thư viện Python-Pandas không thể xử lý. Ký tự thay thế sẽ được thay bằng ký tự "-" (mã Unicode: 45) khi các tệp được kết hợp. Vấn đề này được xử lý ngay lúc kết hợp thay vì sau khi nhập bộ dữ liệu dưới dạng DataFrame vì bộ dữ liệu có kích thước rất lớn và thư viện Pandas không được tối ưu cho thao tác đó.

In [49]:
# Đưa tên tất cả các tệp vào một tuple
dataset = 'CIC-IDS2017'
csv_file_names = ('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX', 
                'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX',
                'Friday-WorkingHours-Morning.pcap_ISCX',
                'Monday-WorkingHours.pcap_ISCX',
                'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX',
                'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX',
                'Tuesday-WorkingHours.pcap_ISCX',
                'Wednesday-workingHours.pcap_ISCX')

In [51]:
# Định nghĩa hàm để kết hợp các tệp riêng lẻ của một bộ dữ liệu
# Ngoài việc kết hợp các tệp, hàm này có hai chức năng bổ sung:
# 1. Thay thế ký tự thay thế (\uFFFD) bằng dấu gạch ngang '-', điều này cần thiết cho bộ dữ liệu CIC-IDS2017
# 2. Giảm kích thước mẫu dữ liệu. Nếu tham số reduce_sample_size được đặt là True, chỉ 10% bộ dữ liệu sẽ được chọn ngẫu nhiên và lưu lại
def combine_csv_files(dataset: str, file_names: tuple, reduce_sample_size: bool = False):

    # Tạo thư mục mới để chứa bộ dữ liệu đã được kết hợp
    os.makedirs('./Dataset/dataset_combined', exist_ok=True)

    # Xóa bộ dữ liệu nếu đã tồn tại
    merged_dataset_directory = f'Dataset/dataset_combined/{dataset}.csv'
    if os.path.isfile(merged_dataset_directory):
        os.remove(merged_dataset_directory)
        print(f'original file({merged_dataset_directory}) has been removed')

    for (file_index, file_name) in enumerate(file_names):
        with open(f"Dataset/{dataset}/{file_name}.csv", 'r') as file, open(merged_dataset_directory, 'a') as out_file:
            for (line_index, line) in enumerate(file):
                
                # Chỉ lấy phần tiêu đề (header) của tệp đầu tiên
                if 'Label' in line or 'label' in line:
                    if file_index != 0 or line_index != 0:
                        continue
                elif reduce_sample_size:
                    if random.randint(1, 10) > 1:
                        continue
                # Thay thế ký tự thay thế (\uFFFD) bằng dấu '-' nếu có   
                out_file.write(line.replace(' ï¿½ ', '-'))
                

In [53]:
combine_csv_files(dataset=dataset, file_names=csv_file_names)

## Bước 2. Phân tích sơ bộ

In [56]:
# Đưa tất cả các tệp CSV vào một DataFrame duy nhất
cic_ids2017 = pd.read_csv('Dataset/dataset_combined/CIC-IDS2017.csv')
cic_ids2017.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
print(f"Số lượng dòng: {cic_ids2017.shape[0]}")
print(f"Số lượng cột: {cic_ids2017.shape[1]}")

Number of rows: 2830743
Number of columns: 79


In [24]:
print("Columns in the dataset:")
cic_ids2017.columns

Columns in the dataset:


Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

Từ phân phối các lớp, chúng ta có thể thấy rõ rằng bộ dữ liệu CIC-IDS2017 có sự mất cân bằng rất lớn. Các mẫu benign chiếm phần lớn trong bộ dữ liệu, trong khi một số mẫu độc hại như tấn công web chỉ chiếm một phần rất nhỏ. 

In [26]:
print('Class distribution:')
cic_ids2017[' Label'].value_counts()

Class distribution:


 Label
BENIGN                      2273097
DoS Hulk                     231073
PortScan                     158930
DDoS                         128027
DoS GoldenEye                 10293
FTP-Patator                    7938
SSH-Patator                    5897
DoS slowloris                  5796
DoS Slowhttptest               5499
Bot                            1966
Web Attack-Brute Force         1507
Web Attack-XSS                  652
Infiltration                     36
Web Attack-Sql Injection         21
Heartbleed                       11
Name: count, dtype: int64

In [30]:
print('Class distribution (normalized):')
cic_ids2017[' Label'].value_counts()/cic_ids2017.shape[0]*100

Class distribution (normalized):


 Label
BENIGN                      80.300366
DoS Hulk                     8.162981
PortScan                     5.614427
DDoS                         4.522735
DoS GoldenEye                0.363615
FTP-Patator                  0.280421
SSH-Patator                  0.208320
DoS slowloris                0.204752
DoS Slowhttptest             0.194260
Bot                          0.069452
Web Attack-Brute Force       0.053237
Web Attack-XSS               0.023033
Infiltration                 0.001272
Web Attack-Sql Injection     0.000742
Heartbleed                   0.000389
Name: count, dtype: float64

### Kiểm tra giá trị null

In [32]:
cic_ids2017_null_count = cic_ids2017.isnull().sum()
cic_ids2017_null_count = cic_ids2017_null_count[cic_ids2017_null_count > 0]
print(f"Rows contain null value: \n{cic_ids2017_null_count}\n")

cic_ids2017_null_count = cic_ids2017_null_count / cic_ids2017.shape[0] * 100
print(f"Rows contain null value (percentage): \n{cic_ids2017_null_count}\n")

Rows contain null value: 
Flow Bytes/s    1358
dtype: int64

Rows contain null value (percentage): 
Flow Bytes/s    0.047973
dtype: float64



### Kiểm tra giá trị vô cực (infinity)

In [36]:
print('Số lượng mẫu chứa giá trị vô cực (infinity)')
np.isinf(cic_ids2017.iloc[:, :-2]).any(axis=1).sum()

Number of samples contains infinity value:


2867

### Kiểm tra các cột chứa giá trị chuỗi

Kiểm tra các cột có kiểu dữ liệu là object, điều này chỉ ra rằng các cột này chứa giá trị chuỗi. Mục tiêu là tìm xem có cột nào chứa cả giá trị số và chữ cái. Các cột như vậy thường bao gồm các giá trị chuỗi như '?' để chỉ ra giá trị thiếu, do đó cần phải làm sạch.

Trong bộ dữ liệu CIC-IDS2017, chỉ có cột Label chứa giá trị chuỗi, do đó không cần phải làm sạch thêm.

In [39]:
cic_ids2017.dtypes[(cic_ids2017.dtypes != 'int64') & (cic_ids2017.dtypes != 'float64')]

Label    object
dtype: object

### Kiểm tra các bản sao

Kiểm tra các cột bị trùng lặp

Mặc dù không có hai cột nào có tên giống nhau, nhưng thực tế có hai cột cho `Fwd Header Length`. Bằng cách kiểm tra thủ công tất cả các cột, chúng ta có thể thấy rằng có một cột tên là `Fwd Header Length` trong khi có một cột khác tên là `Fwd Header Length.1`

In [43]:
# Kiểm tra các cột bị trùng lặp
cic_ids2017.columns[cic_ids2017.columns.value_counts() > 1]

Index([], dtype='object')

In [45]:
# Hiển thị tất cả các cột của bộ dữ liệu để kiểm tra thủ công
cic_ids2017.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

Kiểm tra các dòng trùng lặp

In [48]:
cic_ids2017_duplicates = cic_ids2017[cic_ids2017.duplicated()]
cic_ids2017_duplicates.shape

(308381, 79)

Từ đây, chúng ta có thể thấy rằng bộ dữ liệu CIC-IDS2017 chứa khá nhiều bản ghi bị trùng lặp. Sau khi kiểm tra số lượng bản sao theo từng lớp mẫu, chúng tôi phát hiện rằng các bản sao không cụ thể thuộc về bất kỳ lớp nào. Mặc dù 11% bản sao là khá nhiều, nhưng những bản sao này sẽ được loại bỏ ở bước tiếp theo. Lý do là:
1. Bộ dữ liệu rất lớn, ngay cả khi chúng ta loại bỏ những mẫu này, vẫn còn đủ mẫu để huấn luyện các mô hình.

2. Các bản sao sẽ khiến các mô hình có xu hướng thiên về chúng. 

In [51]:
print(f"{cic_ids2017_duplicates.shape[0]/cic_ids2017.shape[0]*100:.2f}% Tỷ lệ các dòng bị trùng lặp.")

10.89% of rows are duplicates


In [53]:
cic_ids2017_duplicates[' Label'].value_counts()

 Label
BENIGN                    176613
PortScan                   68111
DoS Hulk                   58224
SSH-Patator                 2678
FTP-Patator                 2005
DoS slowloris                411
DoS Slowhttptest             271
Web Attack-Brute Force        37
Bot                           13
DDoS                          11
DoS GoldenEye                  7
Name: count, dtype: int64

## Bước 3. Làm sạch bộ dữ liệu

Từ phân tích ở Bước 2, đã phát hiện rằng bộ dữ liệu chứa một lượng nhỏ bản ghi có giá trị vô cực (infinity) hoặc giá trị thiếu. Những bản ghi này sẽ được loại bỏ vì chúng chỉ chiếm một phần nhỏ trong bộ dữ liệu.

Bên cạnh đó, đã phát hiện rằng 11% bộ dữ liệu là bản sao. Mặc dù các bản sao chiếm một phần lớn trong bộ dữ liệu, nhưng chúng vẫn sẽ được loại bỏ.

Về các cột của bộ dữ liệu, có một cột bị trùng lặp, cột này sẽ bị loại bỏ trong bước này. Ngoài ra, một số tên cột có chứa một khoảng trắng thừa ở đầu tên cột. Khoảng trắng thừa này cũng sẽ được xử lý tại đây.

In [56]:
# Loại bỏ các dòng có giá trị thiếu hoặc giá trị vô cực
# Bằng cách tạm thời xem giá trị vô cực là giá trị null và sử dụng hàm dropna()
with pd.option_context('mode.use_inf_as_na', True):
    cic_ids2017 = cic_ids2017.dropna(how='any')

cic_ids2017.shape

C:\Users\garan\AppData\Local\Temp\ipykernel_11132\169629030.py:3: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


(2827876, 79)

In [58]:
cic_ids2017 = cic_ids2017.drop_duplicates()
cic_ids2017.shape

(2520798, 79)

In [59]:
# Loại bỏ cột bị trùng lặp
cic_ids2017 = cic_ids2017.drop(' Fwd Header Length.1', axis=1)
cic_ids2017.shape

(2520798, 78)

In [60]:
# Loại bỏ khoảng trắng thừa trong tên cột
cic_ids2017_columns = [column for column in cic_ids2017.columns]
for column_index, column in enumerate(cic_ids2017_columns):
    if column[0] == ' ':
        cic_ids2017_columns[column_index] = column[1:]

cic_ids2017.columns = cic_ids2017_columns
cic_ids2017.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

## Bước 4. Chuẩn bị bộ dữ liệu

Trong bước này, mục tiêu chính là giải quyết vấn đề mất cân bằng lớp nghiêm trọng. Để giảm thiểu vấn đề mất cân bằng, các lớp lớn sẽ được giảm mẫu. Bên cạnh đó, tất cả các mẫu tấn công sẽ được gộp lại. Do đó, bộ dữ liệu sẽ chỉ chứa hai nhãn, 'benign' và 'malicious'. Ngoài ra, các mẫu 'benign' sẽ được giảm mẫu sao cho tỷ lệ giữa số lượng mẫu benign và mẫu malicious sẽ là $1:1$

### Giảm mẫu bộ dữ liệu

In [66]:

def downsample_dataset(dataset: pd.DataFrame, sample_count_per_class: pd.Series, max_sample: int) -> pd.DataFrame:
    dataset_downsampled = pd.DataFrame()
    for label, count in sample_count_per_class.items():

        # Đặt giới hạn trên
        if count > max_sample:
            sample_size = max_sample
        else:
            sample_size = count

        sample = dataset[dataset['Label'] == label].sample(n=sample_size).reset_index(drop=True)
        dataset_downsampled = pd.concat([dataset_downsampled, sample])
    
    return dataset_downsampled

In [68]:
# Lấy số lượng mẫu cho mỗi lớp
sample_count_per_class = cic_ids2017['Label'].value_counts()
sample_count_per_class

Label
BENIGN                      2095057
DoS Hulk                     172846
DDoS                         128014
PortScan                      90694
DoS GoldenEye                 10286
FTP-Patator                    5931
DoS slowloris                  5385
DoS Slowhttptest               5228
SSH-Patator                    3219
Bot                            1948
Web Attack-Brute Force         1470
Web Attack-XSS                  652
Infiltration                     36
Web Attack-Sql Injection         21
Heartbleed                       11
Name: count, dtype: int64

In [70]:
cic_ids2017_attack = downsample_dataset(cic_ids2017, sample_count_per_class[1:], max_sample=100000)
cic_ids2017_benign = cic_ids2017[cic_ids2017['Label'] == 'BENIGN'].sample(n=cic_ids2017_attack.shape[0]).reset_index(drop=True)
cic_ids2017_downsampled = pd.concat([cic_ids2017_attack, cic_ids2017_benign])
del cic_ids2017_attack
del cic_ids2017_benign

print('Distribution of class after downsampling')
cic_ids2017_downsampled['Label'].value_counts()

Distribution of class after downsampling


Label
BENIGN                      324881
DoS Hulk                    100000
DDoS                        100000
PortScan                     90694
DoS GoldenEye                10286
FTP-Patator                   5931
DoS slowloris                 5385
DoS Slowhttptest              5228
SSH-Patator                   3219
Bot                           1948
Web Attack-Brute Force        1470
Web Attack-XSS                 652
Infiltration                    36
Web Attack-Sql Injection        21
Heartbleed                      11
Name: count, dtype: int64

In [72]:
print('Class distribution (normalized):')
cic_ids2017_downsampled['Label'].value_counts()/cic_ids2017_downsampled.shape[0]*100

Class distribution (normalized):


Label
BENIGN                      50.000000
DoS Hulk                    15.390251
DDoS                        15.390251
PortScan                    13.958034
DoS GoldenEye                1.583041
FTP-Patator                  0.912796
DoS slowloris                0.828765
DoS Slowhttptest             0.804602
SSH-Patator                  0.495412
Bot                          0.299802
Web Attack-Brute Force       0.226237
Web Attack-XSS               0.100344
Infiltration                 0.005540
Web Attack-Sql Injection     0.003232
Heartbleed                   0.001693
Name: count, dtype: float64

### Gán lại nhãn cho bộ dữ liệu

Sau khi gán lại nhãn, bộ dữ liệu sẽ chỉ còn hai lớp., `malicious` và `benign`

In [75]:
cic_ids2017_downsampled.iloc[cic_ids2017_downsampled['Label'] != 'BENIGN', -1] = 'malicious'
cic_ids2017_downsampled.iloc[cic_ids2017_downsampled['Label'] == 'BENIGN', -1] = 'benign'

cic_ids2017_downsampled['Label'].value_counts()

Label
malicious    324881
benign       324881
Name: count, dtype: int64

## Bước 5. Lưu bộ dữ liệu

In [78]:
# Hàm để lưu bộ dữ liệu đã được làm sạch
def save_cleaned_dataset(dataframe: pd.DataFrame,dataset: str, tag: str = ""):
    # Tạo một thư mục mới để lưu bộ dữ liệu đã được làm sạch.
    os.makedirs('./Dataset/dataset_cleaned', exist_ok=True)

    if not(tag == ""):
        tag = "_" + tag

    dataframe.to_csv(f'Dataset/dataset_cleaned/{dataset}{tag}.csv', index=False)

In [80]:
save_cleaned_dataset(dataframe=cic_ids2017_downsampled, dataset='CIC-IDS2017')

# Bộ Dữ liệu CSE-CIC-IDS2018

Dưới đây là các bước tiền xử lý cho bộ dữ liệu CSE-CIC-IDS2018. Quá trình tổng thể cơ bản giống như bộ dữ liệu CIC-IDS2017. Tuy nhiên, có một số bước xử lý bổ sung cần thực hiện đối với bộ dữ liệu CSE-CIC-IDS2018.

Quá trình bổ sung rõ ràng nhất là cần phải căn chỉnh các cột của bộ dữ liệu CSE-CIC-IDS2018 với bộ dữ liệu CIC-IDS2017. Do đó, chúng ta sẽ cần phải đổi tên các cột trong bộ dữ liệu 2018 và đảm bảo thứ tự các cột hoàn toàn giống nhau ở cả hai bộ dữ liệu.

Hơn nữa, các tệp của bộ dữ liệu 2018 gặp một số vấn đề. Có hai vấn đề chính trong các tệp:

1. Trong một số tệp, có tiêu đề bị trùng lặp trong cùng một tệp.

2. Trong tệp Tuesday-20-02-2018_TrafficForML_CICFlowMeter.csv, có 84 cột thay vì 80 như các tệp khác.

Vấn đề đầu tiên sẽ được giải quyết khi kết hợp các tệp thành một tệp CSV duy nhất bằng cách sử dụng hàm tự định nghĩa combine_csv_files(). Hàm này sẽ giải quyết vấn đề bằng cách bỏ qua tất cả các tiêu đề, chỉ giữ lại dòng tiêu đề đầu tiên của tệp đầu tiên.

Vấn đề thứ hai sẽ được giải quyết trong Bước 0 bằng cách kiểm tra xem bốn cột thừa nào và những cột đó sẽ được loại bỏ. 

## Bước 0. Làm sạch các cột thừa trong tệp Tuesday-20-02-2018_TrafficForML_CICFlowMeter.csv

### Tải một tệp bình thường

Tải một tệp bình thường để chúng ta có thể biết được các cột có trong các tệp có 80 cột.

In [112]:
# Vì mục đích của chúng ta là lấy các cột trong tệp, nên chỉ tải một số lượng nhỏ dòng dữ liệu.
cse_cic_ids2018 = pd.read_csv(f'Dataset/CSE-CIC-IDS2018/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv', nrows=10)
cse_cic_ids2018_typical_columns = pd.Series(cse_cic_ids2018.columns, dtype='str')
print(cse_cic_ids2018.shape)

(10, 80)


### Tải tệp gặp sự cố

In [115]:
cse_cic_ids2018_20022018 = pd.read_csv('Dataset/CSE-CIC-IDS2018/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv', nrows=10)
cse_cic_ids2018_20022018_columns = pd.Series(cse_cic_ids2018_20022018.columns, dtype='str')

cse_cic_ids2018_20022018.shape

(10, 84)

### Kiểm tra các cột chỉ có trong tệp gặp sự cố.

Từ kết quả dưới đây, chúng ta thấy có bốn cột thừa chỉ có trong tệp gặp sự cố. Bên cạnh đó, chỉ số của các cột này cho thấy chúng là bốn cột đầu tiên trong tệp. Vì chỉ có bốn cột thừa, điều này có nghĩa là không có cột nào bị thiếu trong tệp gặp sự cố.

In [118]:
cse_cic_ids2018_20022018_columns[~cse_cic_ids2018_20022018_columns.isin(cse_cic_ids2018_typical_columns)]

0     Flow ID
1      Src IP
2    Src Port
3      Dst IP
dtype: object

### Giải quyết vấn đề

Vì lý do hiệu suất, chúng ta sử dụng thư viện CSV để loại bỏ bốn cột đầu tiên của mỗi dòng và lưu lại vào một tệp CSV mới. Đồng thời, chúng ta cũng sửa lỗi chính tả "Thuesday" thành "Tuesday".

In [121]:
with open('Dataset/CSE-CIC-IDS2018/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv', 'r') as source, \
    open('Dataset/CSE-CIC-IDS2018/Tuesday-20-02-2018_TrafficForML_CICFlowMeter_dropped_first_four_columns.csv', 'w') as result:

    original_dataset = csv.reader(source)
    writer = csv.writer(result)

    for row in original_dataset:
        # exclude the first 4 columns when writing the file
        writer.writerow((row[4:]))

### Tải tệp đã xử lý để đảm bảo rằng vấn đề đã được giải quyết

In [124]:
cse_cic_ids2018_20022018 = pd.read_csv('Dataset/CSE-CIC-IDS2018/Tuesday-20-02-2018_TrafficForML_CICFlowMeter_dropped_first_four_columns.csv', nrows=10)
print(cse_cic_ids2018_20022018.shape)
print(cse_cic_ids2018_20022018.columns)

(10, 80)
Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg

## Bước 1. Kết hợp các tệp của bộ dữ liệu

Khi kết hợp bộ dữ liệu CSE-CIC-IDS2018, chỉ 10% của bộ dữ liệu sẽ được sử dụng. Lý do là bộ dữ liệu 2018 quá lớn để có thể xử lý bởi phần cứng được sử dụng trong công việc này. Bên cạnh đó, bộ dữ liệu CSE-CIC-IDS2018 chỉ được sử dụng cho việc kiểm tra, 10% bộ dữ liệu là đủ.

In [127]:
dataset = 'CSE-CIC-IDS2018'
dataset_csv_files = ('Friday-02-03-2018_TrafficForML_CICFlowMeter',
                    'Friday-16-02-2018_TrafficForML_CICFlowMeter',
                    'Friday-23-02-2018_TrafficForML_CICFlowMeter',
                    'Tuesday-20-02-2018_TrafficForML_CICFlowMeter_dropped_first_four_columns',
                    'Thursday-01-03-2018_TrafficForML_CICFlowMeter',
                    'Thursday-15-02-2018_TrafficForML_CICFlowMeter',
                    'Thursday-22-02-2018_TrafficForML_CICFlowMeter',
                    'Wednesday-14-02-2018_TrafficForML_CICFlowMeter',
                    'Wednesday-21-02-2018_TrafficForML_CICFlowMeter',
                    'Wednesday-28-02-2018_TrafficForML_CICFlowMeter')

In [129]:
combine_csv_files(dataset, dataset_csv_files, reduce_sample_size=True)

## Bước 2. Phân tích sơ bộ

In [132]:
# load the dataset into one DataFrame
cse_cic_ids2018 = pd.read_csv('Dataset/dataset_combined/CSE-CIC-IDS2018.csv')
cse_cic_ids2018.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49684,6,02/03/2018 08:47:38,281,2,1,38.0,0.0,38.0,0.0,...,20,0.0000,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,Benign
1,443,6,02/03/2018 08:47:41,250,2,0,0.0,0.0,0.0,0.0,...,20,0.0000,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,Benign
2,443,6,02/03/2018 08:47:40,60860062,15,13,870.0,3306.0,535.0,0.0,...,20,130201.6667,148831.65440,434003.0,69408.0,10000000.0,16330.68406,10000000.0,9968389.0,Benign
3,443,6,02/03/2018 08:47:38,118281864,36,83,1022.0,108156.0,250.0,0.0,...,20,134521.0000,79961.04903,191062.0,77980.0,59000000.0,44452.97491,59000000.0,58900000.0,Benign
4,49745,6,02/03/2018 08:51:24,96328,2,1,38.0,0.0,38.0,0.0,...,20,0.0000,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,Benign


In [134]:
print(f"Number of rows: {cse_cic_ids2018.shape[0]}")
print(f"Number of columns: {cse_cic_ids2018.shape[1]}")

Number of rows: 1622580
Number of columns: 80


In [136]:
print("Các cột trong bộ dữ liệu:")
cic_ids2017.columns

Columns in the dataset:


Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

Cũng giống như bộ dữ liệu CIC-IDS2017, bộ dữ liệu CSE-CIC-IDS2018 cũng gặp phải vấn đề mất cân bằng lớp nghiêm trọng.

In [139]:
print('Phân phối lớp:')
cse_cic_ids2018['Label'].value_counts()

Class distribution:


Label
Benign                      1347953
DDOS attack-HOIC              68801
DDoS attacks-LOIC-HTTP        57550
DoS attacks-Hulk              46014
Bot                           28539
FTP-BruteForce                19484
SSH-Bruteforce                18485
Infilteration                 16160
DoS attacks-SlowHTTPTest      14110
DoS attacks-GoldenEye          4154
DoS attacks-Slowloris          1076
DDOS attack-LOIC-UDP            163
Brute Force -Web                 59
Brute Force -XSS                 25
SQL Injection                     7
Name: count, dtype: int64

In [141]:
print('Phân phối lớp (đã chuẩn hóa):')
cse_cic_ids2018['Label'].value_counts()/cse_cic_ids2018.shape[0]*100

Class distribution (normalized):


Label
Benign                      83.074671
DDOS attack-HOIC             4.240222
DDoS attacks-LOIC-HTTP       3.546820
DoS attacks-Hulk             2.835854
Bot                          1.758866
FTP-BruteForce               1.200804
SSH-Bruteforce               1.139235
Infilteration                0.995945
DoS attacks-SlowHTTPTest     0.869603
DoS attacks-GoldenEye        0.256012
DoS attacks-Slowloris        0.066314
DDOS attack-LOIC-UDP         0.010046
Brute Force -Web             0.003636
Brute Force -XSS             0.001541
SQL Injection                0.000431
Name: count, dtype: float64

### Kiểm tra giá trị null

In [144]:
cse_cic_ids2018_null_count = cse_cic_ids2018.isnull().sum()
cse_cic_ids2018_null_count = cse_cic_ids2018_null_count[cse_cic_ids2018_null_count > 0]
print(f"Các dòng chứa giá trị null: \n{cse_cic_ids2018_null_count}\n")

cse_cic_ids2018_null_count = cse_cic_ids2018_null_count / cse_cic_ids2018.shape[0] * 100
print(f"Tỷ lệ phần trăm các dòng chứa giá trị null \n{cse_cic_ids2018_null_count}\n")

Rows contain null value: 
Flow Byts/s    5975
dtype: int64

Rows contain null value (percentage): 
Flow Byts/s    0.368241
dtype: float64



### Kiểm tra giá trị vô cực

In [147]:
inf_count = np.isinf(cse_cic_ids2018.iloc[:, 3:-1]).any(axis=1).sum()
print(f'Số lượng dòng chứa giá trị vô cực: {inf_count}; {inf_count/cse_cic_ids2018.shape[0]*100:.2f}% of rows')

Number of rows includes infinity value: 9539; 0.59% of rows


### Kiểm tra các cột chứa giá trị chuỗi

Bộ dữ liệu CSE-CIC-IDS2018 chứa hai cột có kiểu dữ liệu`object`, đó là cột `Timestamp` và `Label`. Vì cả hai cột này không lưu trữ giá trị số, điều này là hoàn toàn bình thường và không cần làm sạch thêm.

In [150]:
cse_cic_ids2018.dtypes[(cse_cic_ids2018.dtypes != 'int64') & (cse_cic_ids2018.dtypes != 'float64')]

Timestamp    object
Label        object
dtype: object

In [152]:
cse_cic_ids2018['Label'].unique()

array(['Benign', 'Bot', 'DoS attacks-SlowHTTPTest', 'DoS attacks-Hulk',
       'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection',
       'DDoS attacks-LOIC-HTTP', 'Infilteration', 'DoS attacks-GoldenEye',
       'DoS attacks-Slowloris', 'FTP-BruteForce', 'SSH-Bruteforce',
       'DDOS attack-LOIC-UDP', 'DDOS attack-HOIC'], dtype=object)

In [154]:
cse_cic_ids2018['Timestamp']


0          02/03/2018 08:47:38
1          02/03/2018 08:47:41
2          02/03/2018 08:47:40
3          02/03/2018 08:47:38
4          02/03/2018 08:51:24
                  ...         
1622575    28/02/2018 01:48:05
1622576    28/02/2018 02:23:17
1622577    28/02/2018 04:05:10
1622578    28/02/2018 11:12:19
1622579    28/02/2018 01:48:05
Name: Timestamp, Length: 1622580, dtype: object

### Kiểm tra các cột bị trùng lặp

Check for duplicated column

In [158]:
cse_cic_ids2018.columns[cse_cic_ids2018.columns.value_counts() > 1]

Index([], dtype='object')

Check for duplicate rows

In [161]:
cse_cic_ids2018_duplicates = cse_cic_ids2018[cse_cic_ids2018.duplicated()]
print(f"number of duplicated rows: {cse_cic_ids2018_duplicates.shape[0]}")
print(f"{cse_cic_ids2018_duplicates.shape[0]/cse_cic_ids2018.shape[0]*100:.2f}% of rows are duplicates")

number of duplicated rows: 17140
1.06% of rows are duplicates


## Bước 3. Làm sạch bộ dữ liệu

Từ phân tích ở Bước 2, đã phát hiện rằng bộ dữ liệu CSE-CIC-IDS2018 chứa một lượng nhỏ các dòng có giá trị thiếu hoặc giá trị vô cực và một lượng nhỏ các bản sao. Những bản ghi này sẽ được loại bỏ vì chúng chỉ chiếm một phần nhỏ trong bộ dữ liệu.

In [164]:
# Loại bỏ các dòng có giá trị null và giá trị vô cực
with pd.option_context('mode.use_inf_as_na', True):
    cse_cic_ids2018 = cse_cic_ids2018.dropna(how='any')

cse_cic_ids2018.shape

C:\Users\garan\AppData\Local\Temp\ipykernel_11132\1842356312.py:2: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


(1613041, 80)

'mode.use_inf_as_na' là một tùy chọn trong pandas, giúp xử lý các giá trị vô cực (infinity) như giá trị thiếu (NaN). Khi tùy chọn này được bật, bất kỳ giá trị vô cực nào trong bộ dữ liệu sẽ được coi là giá trị thiếu và có thể được xử lý bằng các phương pháp xử lý giá trị thiếu như dropna() hoặc fillna().

Cụ thể, khi mode.use_inf_as_na được đặt là True, pandas sẽ tự động coi các giá trị vô cực như NaN trong các phép toán hoặc khi xử lý dữ liệu, giúp dễ dàng xử lý chúng trong quá trình làm sạch và phân tích dữ liệu.

In [166]:
cse_cic_ids2018 = cse_cic_ids2018.drop_duplicates()
cse_cic_ids2018.shape

(1595913, 80)

## Bước 4. Chuẩn bị bộ dữ liệu

Đối với bộ dữ liệu CSE-CIC-IDS2018, có hai mục tiêu trong bước này:

1. Giống như bộ dữ liệu CIC-IDS2017, bộ dữ liệu 2018 sẽ được giảm mẫu và tất cả các mẫu tấn công sẽ được gán nhãn là 'malicious'.

2. Các tên cột của bộ dữ liệu 2018 sẽ được sửa đổi để khớp với bộ dữ liệu 2017.

### Giảm mẫu bộ dữ liệu

In [ ]:
# Lấy số lượng mẫu cho mỗi lớp
sample_count_per_class = cse_cic_ids2018['Label'].value_counts()
sample_count_per_class

In [172]:
ids2018_attack = downsample_dataset(cse_cic_ids2018, sample_count_per_class[1:], max_sample=100000)
num_attack_sample = ids2018_attack.shape[0]
ids2018_benign = cse_cic_ids2018[cse_cic_ids2018['Label'] == 'Benign'].sample(n=num_attack_sample).reset_index(drop=True)
ids2018_downsampled = pd.concat([ids2018_attack, ids2018_benign])
del ids2018_attack
del ids2018_benign

print('Phân phối lớp sau khi giảm mẫu')
ids2018_downsampled['Label'].value_counts()

Distribution of class after downsampling


Label
Benign                      257791
DDOS attack-HOIC             68628
DDoS attacks-LOIC-HTTP       57550
DoS attacks-Hulk             45691
Bot                          28501
SSH-Bruteforce               16312
Infilteration                16034
FTP-BruteForce               12368
DoS attacks-SlowHTTPTest      7251
DoS attacks-GoldenEye         4153
DoS attacks-Slowloris         1049
DDOS attack-LOIC-UDP           163
Brute Force -Web                59
Brute Force -XSS                25
SQL Injection                    7
Name: count, dtype: int64

In [174]:
print('Phân phối lớp (đã chuẩn hóa):')
ids2018_downsampled['Label'].value_counts()/ids2018_downsampled.shape[0]*100

Class distribution (normalized):


Label
Benign                      50.000000
DDOS attack-HOIC            13.310783
DDoS attacks-LOIC-HTTP      11.162143
DoS attacks-Hulk             8.862024
Bot                          5.527928
SSH-Bruteforce               3.163803
Infilteration                3.109884
FTP-BruteForce               2.398842
DoS attacks-SlowHTTPTest     1.406372
DoS attacks-GoldenEye        0.805497
DoS attacks-Slowloris        0.203459
DDOS attack-LOIC-UDP         0.031615
Brute Force -Web             0.011443
Brute Force -XSS             0.004849
SQL Injection                0.001358
Name: count, dtype: float64

### Gán lại nhãn cho bộ dữ liệu

In [ ]:
# Thay thế nhãn của tất cả các lớp tấn công thành 'malicious'
# Thay thế nhãn của tất cả các lớp an toàn thành 'benign'
ids2018_downsampled.iloc[ids2018_downsampled['Label'] != 'Benign', -1] = 'malicious'
ids2018_downsampled.iloc[ids2018_downsampled['Label'] == 'Benign', -1] = 'benign'
ids2018_downsampled['Label'].value_counts()

### Căn chỉnh tên cột với bộ dữ liệu CIC-IDS2017

In [180]:
ids2018_columns = pd.Series(ids2018_downsampled.columns, dtype='str')
ids2017_columns = pd.Series(cic_ids2017.columns, dtype='str')

In [182]:
column_mapping = {
    "Dst": "Destination",
    "TotLen": "Total Length",
    "Tot": "Total", 
    "Pkt": "Packet",
    "Len": "Length",
    "Cnt": "Count", 
    "Var": "Variance",
    "Total Bwd Packets": "Total Backward Packets", 
    "Totalal Lengthgth Fwd Packets": "Total Length of Fwd Packets", 
    "Totalal Lengthgth Bwd Packets": "Total Length of Bwd Packets", 
    "Flow Byts/s": "Flow Bytes/s",
    "Packet Size Avg": "Average Packet Size",
    "Fwd Seg Size Avg": "Avg Fwd Segment Size",
    "Bwd Seg Size Avg": "Avg Bwd Segment Size",
    "Fwd Byts/b Avg": "Fwd Avg Bytes/Bulk",
    "Fwd Packets/b Avg": "Fwd Avg Packets/Bulk",
    "Fwd Blk Rate Avg": "Fwd Avg Bulk Rate", 
    "Bwd Byts/b Avg": "Bwd Avg Bytes/Bulk",
    "Bwd Packets/b Avg": "Bwd Avg Packets/Bulk",
    "Bwd Blk Rate Avg": "Bwd Avg Bulk Rate", 
    "Init Fwd Win Byts": "Init_Win_bytes_forward", 
    "Init Bwd Win Byts": "Init_Win_bytes_backward",
    "Fwd Act Data Packets": "act_data_pkt_fwd",
    "Fwd Seg Size Min": "min_seg_size_forward",
    "Subflow Fwd Byts": "Subflow Fwd Bytes", 
    "Subflow Bwd Byts": "Subflow Bwd Bytes"
}

In [184]:
# Đổi tên các cột của bộ dữ liệu 2018
for original_value in column_mapping:
    ids2018_columns = ids2018_columns.replace({original_value: column_mapping[original_value]}, regex=True)

ids2018_columns[40] = 'Min Packet Length'
ids2018_columns[41] = 'Max Packet Length'

Kiểm tra các cột thừa trong bộ dữ liệu CSE-CIC-IDS2018

In [187]:
ids2018_columns[~ids2018_columns.isin(ids2017_columns)]

1     Protocol
2    Timestamp
dtype: object

Kiểm tra xem bộ dữ liệu CSE-CIC-IDS2018 có thiếu cột nào có trong bộ dữ liệu CIC-IDS2017 hay không

In [190]:
ids2017_columns[~ids2017_columns.isin(ids2018_columns)]

Series([], dtype: object)

Thay đổi tên cột của DataFrame và loại bỏ các cột bổ sung

In [193]:
ids2018_downsampled.columns = ids2018_columns
# drop the additional columns
ids2018_downsampled = ids2018_downsampled.drop(['Protocol', 'Timestamp'], axis=1).copy()
ids2018_downsampled.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,1794,3,4,309.0,935.0,309.0,0.0,103.000000,178.401233,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,malicious
1,80,1205,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,malicious
2,80,18083,3,4,298.0,935.0,298.0,0.0,99.333333,172.050380,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,malicious
3,80,23013,2,0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,malicious
4,80,17255,3,4,326.0,935.0,326.0,0.0,108.666667,188.216188,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,malicious


### Đảm bảo thứ tự các cột hoàn toàn giống nhau ở cả hai bộ dữ liệu

In [196]:
ids2018_downsampled = ids2018_downsampled.reindex(columns=ids2017_columns)

## Bước 5. Lưu bộ dữ liệu

In [199]:
save_cleaned_dataset(ids2018_downsampled, 'CSE-CIC-IDS2018')